In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import darts

In [2]:
climate_data = pd.read_excel('2025 Allianz Datathon Dataset.xlsx', sheet_name='Climate Data')
visitation_data = pd.read_excel('2025 Allianz Datathon Dataset.xlsx', sheet_name='Visitation Data')
snow_depth_nsw = pd.read_csv('snow_depth_dataset.csv')

In [3]:
climate_data['Date'] = pd.to_datetime(climate_data[['Year', 'Month', 'Day']])

In [4]:
climate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39813 entries, 0 to 39812
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Bureau of Meteorology station number  39813 non-null  int64         
 1   Year                                  39813 non-null  int64         
 2   Month                                 39813 non-null  int64         
 3   Day                                   39813 non-null  int64         
 4   Maximum temperature (Degree C)        38275 non-null  float64       
 5   Minimum temperature (Degree C)        38280 non-null  float64       
 6   Rainfall amount (millimetres)         37857 non-null  float64       
 7   Date                                  39813 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(4)
memory usage: 2.4 MB


In [5]:
# Filter Months exclusive to winter
climate_data = climate_data[climate_data['Month'].between(6, 9, inclusive='both')]
climate_data = climate_data.drop(['Month', 'Day'], axis=1)
# Adjust based on Ski Season Dates
climate_data = climate_data[(
   ( climate_data['Date'].dt.month > 6) | ((climate_data['Date'].dt.month == 6) & (climate_data['Date'].dt.day >= 9))
   & 
   ( climate_data['Date'].dt.month < 9) | ((climate_data['Date'].dt.month == 9) & (climate_data['Date'].dt.day >= 15))
)]

In [6]:
climate_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12317 entries, 159 to 39812
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Bureau of Meteorology station number  12317 non-null  int64         
 1   Year                                  12317 non-null  int64         
 2   Maximum temperature (Degree C)        11896 non-null  float64       
 3   Minimum temperature (Degree C)        11910 non-null  float64       
 4   Rainfall amount (millimetres)         11859 non-null  float64       
 5   Date                                  12317 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 673.6 KB


In [7]:
climate_data[climate_data.isnull().any(axis=1)]

,Bureau of Meteorology station number,Year,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres),Date
571,71075,2011,1.7,NaN,NaN,2011-07-26
581,71075,2011,6.8,4.2,NaN,2011-08-05
2386,71075,2016,NaN,-7.0,4.2,2016-07-14
2387,71075,2016,NaN,-4.4,NaN,2016-07-15
2388,71075,2016,10.3,-3.9,NaN,2016-07-16
...,...,...,...,...,...,...
39113,72161,2023,11.5,NaN,NaN,2023-08-29
39484,72161,2024,NaN,-3.5,2.6,2024-09-03
39485,72161,2024,11.6,-0.3,NaN,2024-09-04
39777,72161,2025,NaN,2.7,0.0,2025-06-23


In [8]:
SKI_WEEK_STARTS = {
     1: (6,  9),  2: (6, 16),  3: (6, 23),  4: (6, 30),
     5: (7,  7),  6: (7, 14),  7: (7, 21),  8: (7, 28),
     9: (8,  4), 10: (8, 11), 11: (8, 18), 12: (8, 25),
    13: (9,  1), 14: (9,  8), 15: (9, 15),
}

def get_ski_week(dt: pd.Timestamp):
    y = dt.year
    for w, (m, d) in SKI_WEEK_STARTS.items():
        start = pd.Timestamp(year=y, month=m, day=d)
        end   = start + pd.Timedelta(days=7)
        if start <= dt < end:
            return w
    return pd.NA  

climate_data['Week'] = climate_data['Date'].apply(get_ski_week)
climate_data = climate_data.dropna(subset=['Week']).copy()
climate_data['Week'] = climate_data['Week'].astype('int64')


In [9]:
# Fill NaN with mean value of respective column
climate_data = climate_data.fillna(climate_data.mean(numeric_only=True))

In [10]:
climate_data.head(20)

,Bureau of Meteorology station number,Year,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres),Date,Week
159,71075,2010,-2.3,-3.8,0.0,2010-06-09,1
160,71075,2010,2.2,-4.6,12.4,2010-06-10,1
161,71075,2010,2.0,-2.9,0.0,2010-06-11,1
162,71075,2010,3.5,-6.0,0.2,2010-06-12,1
163,71075,2010,8.3,-10.3,0.0,2010-06-13,1
164,71075,2010,7.1,-10.8,0.0,2010-06-14,1
165,71075,2010,2.4,-6.0,0.0,2010-06-15,1
166,71075,2010,4.6,0.7,0.0,2010-06-16,2
167,71075,2010,2.0,0.9,58.0,2010-06-17,2
168,71075,2010,1.1,-3.0,37.6,2010-06-18,2


In [11]:
station_map = {
    71032: ['Thredbo', 'Charlotte Pass'],
    71075: ['Perisher'],
    72161: ['Selwyn'],
    83024: ['Mt. Buller', 'Mt. Stirling'],
    83085: ['Mt. Hotham'],
    85291: ['Mt. Baw Baw'],
    83084: ['Falls Creek']
}

In [12]:
climate_expanded = []

for station, destinations in station_map.items():
    df_station = climate_data[climate_data['Bureau of Meteorology station number'] == station]
    for destination in destinations:
        df_temp = df_station.copy()
        df_temp['Destination'] = destination
        climate_expanded.append(df_temp)

climate_df_expanded = pd.concat(climate_expanded, ignore_index=True)


In [13]:
visitation = visitation_data.melt(
    id_vars=['Year', 'Week'],
    var_name='Destination',
    value_name='Visitors'
)

In [14]:
merged_df = pd.merge(
    visitation,
    climate_df_expanded,
    on=['Year', 'Week', 'Destination'],
    how='inner'
)

In [15]:
snow_depth_nsw.head()

,Location,Year,Week,Date,SnowDepth(cm)
0,Falls Creek,2020,1,9-Jun,27
1,Falls Creek,2020,2,16-Jun,22
2,Falls Creek,2020,3,23-Jun,58
3,Falls Creek,2020,4,30-Jun,54
4,Falls Creek,2020,5,7-Jul,74


In [16]:
snow_depth_nsw['Location'].unique()

array(['Falls Creek', 'Mt Baw Baw', 'Mt Buller', 'Mt Hotham',
       'Mt Stirling', 'Spencers Creek', 'Three Mile Dam'], dtype=object)

In [17]:
merged_df['Destination'].unique()

array(['Mt. Baw Baw', 'Mt. Stirling', 'Mt. Hotham', 'Falls Creek',
       'Mt. Buller', 'Selwyn', 'Thredbo', 'Perisher', 'Charlotte Pass'],
      dtype=object)

In [18]:
# Mapping between snow locations and destinations
location_map = {
    "Spencers Creek": ["Thredbo", "Perisher", "Charlotte Pass"],
    "Three Mile Dam": ["Selwyn"],
    "Falls Creek" : ["Falls Creek"],
    "Mt Baw Baw" : ["Mt. Baw Baw"],
    "Mt Stirling" : ["Mt. Stirling"],
    "Mt Hotham" : ["Mt. Hotham"],
    "Mt Buller" : ["Mt. Buller"],
}

# Reverse the mapping: destination -> snow location
destination_to_location = {
    dest: loc
    for loc, dests in location_map.items()
    for dest in dests
}

# Add snow location column to visitors (merged_df) based on Destination
merged_df["SnowLocation"] = merged_df["Destination"].map(destination_to_location)

# Merge on Year + Week + SnowLocation
complete_df = pd.merge(
    merged_df, snow_depth_nsw,
    left_on=["Year", "Week", "SnowLocation"],
    right_on=["Year", "Week", "Location"],
    how="left"
)

# # Drop duplicate Location columns if you want
complete_df = complete_df.drop(columns=["Date_y", "Location", "SnowLocation"], axis=1)


In [19]:
complete_df = complete_df[complete_df['Date_x'].dt.year >= 2020]
complete_df['SnowDepth(cm)'] = complete_df['SnowDepth(cm)'].fillna(0)

In [20]:
complete_df.head()

,Year,Week,Destination,Visitors,Bureau of Meteorology station number,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres),Date_x,SnowDepth(cm)
630,2020,1,Mt. Baw Baw,1074,85291,6.8,1.000000,4.9975,2020-06-09,0.0
631,2020,1,Mt. Baw Baw,1074,85291,7.4,1.400000,0.0000,2020-06-10,0.0
632,2020,1,Mt. Baw Baw,1074,85291,3.8,-2.154507,4.9975,2020-06-11,0.0
633,2020,1,Mt. Baw Baw,1074,85291,7.2,0.100000,0.0000,2020-06-12,0.0
634,2020,1,Mt. Baw Baw,1074,85291,4.7,-2.154507,0.0000,2020-06-13,0.0


In [21]:
# Split dataset
train_df = complete_df[complete_df["Year"] <= 2023]
val_df   = complete_df[complete_df["Year"] == 2024]
test_df  = complete_df[complete_df["Year"] >= 2025]


drop_cols = ['Week'] 
# Drop Dependent Variable
X_train = train_df.drop(columns=drop_cols)
X_valid = val_df.drop(columns=drop_cols)
X_test  = test_df.drop(columns=drop_cols)

# One hot encode categorical
cat_cols = ['Destination']
X_train = pd.get_dummies(X_train, columns=cat_cols, drop_first=False)
X_valid = pd.get_dummies(X_valid, columns=cat_cols, drop_first=False)
X_test  = pd.get_dummies(X_test,  columns=cat_cols, drop_first=False)

# Align columns across splits
X_valid = X_valid.reindex(columns=X_train.columns, fill_value=0)
X_test  = X_test.reindex(columns=X_train.columns,  fill_value=0)

y_train, y_val, y_test = train_df['Week'].astype(int), val_df['Week'].astype(int), test_df['Week'].astype(int)


In [22]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

xgb = XGBClassifier(
    objective='multi:softprob',
    num_class=15,            # weeks 1..15
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    tree_method="hist"       # fast; if GPU is available: "gpu_hist"
)

xgb.fit(
    X_train.drop(columns=['Year']), y_train,           # drop Year at fit time if you prefer
    eval_set=[(X_valid.drop(columns=['Year']), y_val)],
    eval_metric='mlogloss',
    verbose=False
)

pred_xgb = xgb.predict(X_test.drop(columns=['Year']))
print("XGB Accuracy:", accuracy_score(y_test, pred_xgb))
print("XGB Macro-F1:", f1_score(y_test, pred_xgb, average='macro'))
print(classification_report(y_test, pred_xgb, digits=3))


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'eval_metric'